In [ ]:
import pandas as pd
import numpy as np
from numpy import loadtxt
from pylab import *
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, HTML
import pandas as pd
from scipy import stats
import pandas_profiling
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import missingno as msno
import pickle
from sklearn.preprocessing import LabelEncoder

In [ ]:
url = "https://raw.githubusercontent.com/diegojeda/Curso-Analytics---Drilling-Bits/master/Dataset/BLCat.csv"

base = pd.read_csv(url,error_bad_lines=False)

In [ ]:
# load the model from disk
loaded_model = pickle.load(open('svr_to_deploy.pkl', 'rb'))

In [ ]:
feature_cols=base.drop('Log ROP', axis=1).columns

In [ ]:
feature_cols

In [ ]:
range(100,500,1)

In [ ]:
from itertools import product

Size=[8.5]
Re_Run=["N"]
Type=["PDC"]
Structure=["Anticlinal P"]
Formation=["Mirador"]
Footage=[150]
Depth_Average=[15000]
Blades = [13]
Cutter_size=[11]
Min_FR= list(np.linspace(100,500,num=5))
Max_FR = list(np.linspace(500,1100,num=5))
TFA = list(np.linspace(1,5,num=5))
HSI = list(np.linspace(1,5,num=5))
Min_WOB = list(np.linspace(5, 30, num=6))
Max_WOB = list(np.linspace(30,55,num=6))
Mud_Weight=[10]
Drive_System=["RSS"]
Min_RPM = list(np.linspace(60,120,num=6))
Max_RPM = list(np.linspace(120,170,num=7))
T_Min =  list(np.linspace(5,25,num=5))
T_Max  = list(np.linspace(25,50,num=6))
Manufacturer=["Smith"]

Data_exp=pd.DataFrame(list(product(Size,Re_Run,Type,Structure,Formation,Footage,Depth_Average,Blades,Cutter_size,Min_FR, Max_FR,TFA,HSI,Min_WOB,Max_WOB,Mud_Weight,Drive_System,Min_RPM,Max_RPM,T_Max,T_Min,Manufacturer)),
                 columns=["Size","Re_Run","Type","Structure","Formation","Footage","Depth Average","Blades","Cutter size","Min Flow Rate", "Max Flow Rate","TFA","HSI","Min WOB","Max WOB","Mud Weight","Drive_System","Min_RPM","Max_RPM","Torque Max","Torque Min","Manufacturer"])

Data_exp.iloc[0:10] # Primeras cinco filas

In [ ]:
Base_Frame = Data_exp.copy()
Base_Frame.columns

In [ ]:
Data_exp

In [ ]:
Data_exp.Footage=np.log(Data_exp.Footage)

Data_exp["Min WOB"]=np.log(Data_exp["Min WOB"])

Data_exp = Data_exp.rename(columns = {"Footage":"Log Footage",
                              "Min WOB":"Log Min WOB"})

Data_exp.tail()

In [ ]:
base = base.drop(['Log ROP',"Depth out","Depth in","Bit_Number","Reference","Well","Unnamed: 0"],axis=1)

In [ ]:
base

In [ ]:
base_exp_pred=base.append(Data_exp)

In [ ]:
base_exp_pred

In [ ]:
# Verificamos las categorias de cada variable categorica
print(base_exp_pred.Re_Run.unique())
print(base_exp_pred.Type.unique())
print(base_exp_pred.Structure.unique())
print(base_exp_pred.Formation.unique())
print(base_exp_pred.Drive_System.unique())
print(base_exp_pred.Manufacturer.unique())

In [ ]:
# Identificamos las etiquetas de las categorias
categorical_features = ['Re_Run', 'Type', 'Structure',"Formation","Drive_System","Manufacturer"]

le = LabelEncoder()

# Convert the variables to numerical
for i in range(6):
    new = le.fit_transform(base_exp_pred[categorical_features[i]])
    base_exp_pred[categorical_features[i]] = new

In [ ]:
base_exp_pred.tail()

In [ ]:
# Importamos librerias para escalar los datos, dividir el dataset y traer el modelo
from sklearn.preprocessing import StandardScaler

# Escalamos los datos
scaler = StandardScaler()
scaler.fit(base_exp_pred)
base_exp_pred = scaler.transform(base_exp_pred)
base_exp_pred

In [ ]:
len(Data_exp)

Data_List_Pred=base_exp_pred[-(len(Data_exp)+1):-1]

In [ ]:
y_pred_rfs = loaded_model.predict(Data_List_Pred)
R_final = pd.DataFrame(np.exp(y_pred_rfs))
R_final.columns = ["ROP"]
#R_final
Max_ROP=max(np.exp(y_pred_rfs))
Indice=R_final.loc[R_final.ROP==Max_ROP].index
#Max_ROP
#Indice
P_Opt=Data_exp.iloc[Indice].T
#P_Opt

In [ ]:
P_Opt=Data_exp.iloc[Indice].T

In [ ]:
P_Opt

print("La ROP Max alcanzada con estos parametros es: %.3f (ft/hr)" %(Max_ROP))

In [ ]:
Base_Frame.shape

In [ ]:
y_pred_rfs.shape

In [ ]:
Data_List_Pred.shape

In [ ]:
Base_Frame["ROP Predict"] = np.exp(y_pred_rfs)

In [ ]:
Base_Frame

In [ ]:
Base_Frame[Base_Frame["ROP Predict"]==Max_ROP].T

In [ ]:
Base_Frame.columns

In [ ]:
Test=Base_Frame[(Base_Frame["Min Flow Rate"]==500)&
                (Base_Frame["Max Flow Rate"]==1100)&
                (Base_Frame["TFA"]==4)&
                (Base_Frame["HSI"]==3)&
                (Base_Frame["Min WOB"]==10)&
                (Base_Frame["Max WOB"]==30)&
                (Base_Frame["Min_RPM"]==84)&
                (Base_Frame["Max_RPM"]==120)#&
                #(Base_Frame["Torque Min"]==15)&
                #(Base_Frame["Torque Max"]==50)
               ]
Test

In [ ]:
Tabla=Test.pivot(index="Torque Min",columns="Torque Max",values="ROP Predict")
print(Tabla)

In [ ]:
Test["Min Flow Rate"].max()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

title = "Curva de Nivel ROP"

plt.title(title,fontsize=25)
ttl = ax.title
ttl.set_position([0.5,1.05])

sns.heatmap(Tabla,xticklabels=1,yticklabels=1,cmap="seismic", ax=ax,
            cbar_kws={"shrink": 0.8, "label":" ROP (ft/hr)","orientation": "horizontal"},
            annot_kws={'size':5}
           )
plt.ylim(0,5)
#plt.xlim(1,60);

# Seleccion de parametros optimos de para mapas de calor

In [ ]:
print(range(100,500,10))

In [ ]:
from itertools import product

Size=[8.5]
Re_Run=["N"]
Type=["PDC"]
Structure=["Anticlinal P"]
Formation=["Mirador"]
Footage=[150]
Depth_Average=[15000]
Blades = [13]
Cutter_size=[11]
Min_FR= range(400,710,10) # list(np.linspace(100,500,num=5))
Max_FR = range(800,1200,10) # list(np.linspace(500,1100,num=5))
TFA = [4] # list(np.linspace(1,5,num=5))
HSI = [3] #  list(np.linspace(1,5,num=5))
Min_WOB = [10] # list(np.linspace(5, 30, num=6))
Max_WOB = [30] # list(np.linspace(30,55,num=6))
Mud_Weight=[10]
Drive_System=["RSS"]
Min_RPM = [84] # list(np.linspace(60,120,num=6))
Max_RPM = [120] # list(np.linspace(120,170,num=7))
T_Min = [15] # list(np.linspace(5,25,num=5))
T_Max  = [50] # list(np.linspace(25,50,num=6))
Manufacturer=["Smith"]

Data_exp_1=pd.DataFrame(list(product(Size,Re_Run,Type,Structure,Formation,Footage,Depth_Average,Blades,Cutter_size,Min_FR, Max_FR,TFA,HSI,Min_WOB,Max_WOB,Mud_Weight,Drive_System,Min_RPM,Max_RPM,T_Max,T_Min,Manufacturer)),
                 columns=["Size","Re_Run","Type","Structure","Formation","Footage","Depth Average","Blades","Cutter size","Min Flow Rate", "Max Flow Rate","TFA","HSI","Min WOB","Max WOB","Mud Weight","Drive_System","Min_RPM","Max_RPM","Torque Max","Torque Min","Manufacturer"])

Data_exp_1.iloc[0:10] # Primeras cinco filas

In [ ]:
Base_Frame_1 = Data_exp_1.copy()
Base_Frame_1.columns

In [ ]:
Data_exp_1

In [ ]:
Data_exp_1.Footage=np.log(Data_exp_1.Footage)

Data_exp_1["Min WOB"]=np.log(Data_exp_1["Min WOB"])

Data_exp_1 = Data_exp_1.rename(columns = {"Footage":"Log Footage",
                              "Min WOB":"Log Min WOB"})

Data_exp_1.tail()

In [ ]:
base_exp_pred_1=base.append(Data_exp_1)
base_exp_pred_1

In [ ]:
base_exp_pred_1[(base_exp_pred_1["Min Flow Rate"]==500)&(base_exp_pred_1["Max Flow Rate"]==1100)].T

In [ ]:
# Verificamos las categorias de cada variable categorica
print(base_exp_pred_1.Re_Run.unique())
print(base_exp_pred_1.Type.unique())
print(base_exp_pred_1.Structure.unique())
print(base_exp_pred_1.Formation.unique())
print(base_exp_pred_1.Drive_System.unique())
print(base_exp_pred_1.Manufacturer.unique())

In [ ]:
# Identificamos las etiquetas de las categorias
categorical_features = ['Re_Run', 'Type', 'Structure',"Formation","Drive_System","Manufacturer"]

le = LabelEncoder()

# Convert the variables to numerical
for i in range(6):
    new = le.fit_transform(base_exp_pred_1[categorical_features[i]])
    base_exp_pred_1[categorical_features[i]] = new

base_exp_pred_1.tail()

In [ ]:
# Importamos librerias para escalar los datos, dividir el dataset y traer el modelo
from sklearn.preprocessing import StandardScaler

# Escalamos los datos
scaler = StandardScaler()
scaler.fit(base_exp_pred_1)
base_exp_pred_1 = scaler.transform(base_exp_pred_1)
base_exp_pred_1

In [ ]:
len(Data_exp_1)

Data_List_Pred_1=base_exp_pred_1[-(len(Data_exp_1)+1):-1]

In [ ]:
y_pred_rfs_1 = loaded_model.predict(Data_List_Pred_1)
np.exp(y_pred_rfs_1.max())

In [ ]:
Base_Frame_1

In [ ]:
np.exp(y_pred_rfs_1).size

In [ ]:
Base_Frame_1["ROP Predict"] = np.exp(y_pred_rfs_1)
Base_Frame_1

In [ ]:
base_exp_pred_1[(base_exp_pred_1["Min Flow Rate"]==500)&(base_exp_pred_1["Max Flow Rate"]==1100)].T

In [ ]:
Base_Frame_1.columns

In [ ]:
Tabla_1=Base_Frame_1.pivot(index="Min Flow Rate",columns="Max Flow Rate",values="ROP Predict")
print(Tabla_1)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

title = "Curva de Nivel ROP"

plt.title(title,fontsize=25)
ttl = ax.title
ttl.set_position([0.5,1.05])

sns.heatmap(Tabla_1,xticklabels=10,yticklabels=10,cmap="seismic", ax=ax,
            cbar_kws={"shrink": 0.8, "label":" ROP (ft/hr)","orientation": "horizontal"},
            annot_kws={'size':5}
           )
#plt.ylim(0,40)
#plt.xlim(1,60);